# This notebook explores data on congress expenditures

### A. Loop through all csv files and append into one large dataframe

In [ ]:
import csv
import pandas as pd
df=pd.DataFrame()
path = '/resources/TDI/'
for i in range(2009,2019):
    for j in range(1,5):
        if i == 2015 & j == 2:
            filename = str(i)+'Q'+str(j)+'-house-disburse-detail-updated.csv'
        else:
            filename = str(i)+'Q'+str(j)+'-house-disburse-detail.csv'           
        try:
            dfi = pd.read_csv(path+filename, low_memory=False)
            dfi['file']=filename
            #cols = ['OFFICE', 'QUARTER', 'CATEGORY', 'DATE', 'PAYEE','START DATE', 'END DATE', 'PURPOSE', 'AMOUNT', 'YEAR']
            #dfi = dfi[cols]
            frames = [df, dfi]
            df = pd.concat(frames)
            print(filename)
        except:    
            print(i,'Q',j,' = error')
df1 = pd.read_csv('/resources/TDI/2017Q2-house-disburse-detail.csv',low_memory=False,encoding="ISO-8859-1")
df1['file']='2017Q2-house-disburse-detail.csv'
df2 = pd.read_csv('/resources/TDI/2018Q1-house-disburse-detail.csv',low_memory=False,encoding="ISO-8859-1")
df2['file']='2018Q1-house-disburse-detail.csv'
df3 = pd.read_csv('/resources/TDI/2015Q2-house-disburse-detail-updated.csv')
df3['file']='2015Q2-house-disburse-detail-updated.csv'
frames = [df,df1,df2,df3]
df = pd.concat(frames)
df.to_csv('/resources/TDI/CongressData.csv', sep=',')

### B. Import supporting packages

In [226]:
import csv
import pandas as pd
import numpy as np
import re
import io
import os 
import datetime as DT
import statistics
import datetime
import sys
import types
import pip
from datetime import date
import time
import re
from time import gmtime, strftime

### C. Import csv file for analysis

In [227]:
# The code was removed by Watson Studio for sharing.

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,4,8,9,10,11,12,13,15,16,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,AMOUNT,BIOGUIDE_ID,CATEGORY,DATE,END DATE,OFFICE,PAYEE,PROGRAM,PURPOSE,...,START DATE,TRANSCODE,TRANSCODELONG,YEAR,file,START MONTH,START DAY,START YEAR,START_DATE,START_DATETIME
0,0,NaN,NaN,OTHER SERVICES,NaN,10/04/06,COMMUNICATIONS,07Â­01 P2 OPR0900726A STR...,NaN,NON-TECHNOLOGY SERVICE CONTRCT,...,10/04/06,NaN,NaN,FISCAL YEAR 2009,2009Q3-house-disburse-detail.csv,10,4.0,6.0,10/04/06,2006-10-04
1,1,NaN,NaN,OTHER SERVICES,NaN,10/04/06,COMMUNICATIONS,07Â­22 P2 OPR0900726B ...,NaN,NON-TECHNOLOGY SERVICE CONTRCT,...,10/04/06,NaN,NaN,FISCAL YEAR 2009,2009Q3-house-disburse-detail.csv,10,4.0,6.0,10/04/06,2006-10-04
2,2,NaN,NaN,OTHER SERVICES,NaN,07/18/06,COMMUNICATIONS,08Â­06 P2 FSS0000575A THE...,NaN,NON-TECHNOLOGY SERVICE CONTRCT,...,07/18/06,NaN,NaN,FISCAL YEAR 2009,2009Q3-house-disburse-detail.csv,7,18.0,6.0,07/18/06,2006-07-18
3,3,888.00,NaN,OTHER SERVICES,NaN,05/29/09,COMMUNICATIONS,08Â­25 P2 MFP0003163 AVAYA,NaN,NON-TECHNOLOGY SERVICE CONTRCT,...,05/29/09,NaN,NaN,FISCAL YEAR 2009,2009Q3-house-disburse-detail.csv,5,29.0,9.0,05/29/09,2009-05-29
4,4,590.18,NaN,OTHER SERVICES,NaN,10/04/06,COMMUNICATIONS,09Â­10 P2 OPR0900726C STR...,NaN,NON-TECHNOLOGY SERVICE CONTRCT,...,10/04/06,NaN,NaN,FISCAL YEAR 2009,2009Q3-house-disburse-detail.csv,10,4.0,6.0,10/04/06,2006-10-04


### D. Analysis

### Q1: What is the total of all the payments in the dataset?

In [3]:
df.AMOUNT.sum()

4608689398.1000004

### Q2 Define the 'COVERAGE PERIOD' for each payment as the difference (in days) between 'END DATE' and 'START DATE'. What is the standard deviation in 'COVERAGE PERIOD'? Only consider payments with strictly positive amounts.

a. Format date variables

In [ ]:
def dateVars(datevar, prefix):
    monthvar = prefix+' MONTH'
    dayvar = prefix+' DAY'
    yearvar = prefix+' YEAR'
    
    df[monthvar] = df[datevar].str.split('/').str[0]
    df[dayvar] = df[datevar].str.split('/').str[1]
    df[yearvar] = df[datevar].str.split('/').str[2]
    
    df[monthvar] = df[monthvar].str.zfill(2)
    df[dayvar] = df[dayvar].str.zfill(2)
    
    for i in range(5,22):
        yr = str(i)
        yr = yr.zfill(2)
        df.loc[df[yearvar] == '20'+yr, yearvar] = yr
    
    datevarnew = prefix+'_DATE'
    df[datevarnew]=df[monthvar]+'/'+df[dayvar]+'/'+df[yearvar]
    
    datevartype = prefix+'_DATETIME'
    df[datevartype] = pd.to_datetime(pd.Series(df[datevarnew]), format = '%m/%d/%y', errors='coerce')

dateVars('END DATE', 'END')
dateVars('START DATE', 'START')

b. Generate variable 'COVERAGE PERIOD' as the different between end and start date

In [ ]:
df['COVERAGE DAYS'] = df['END_DATETIME'] - df['START_DATETIME']
df['COVERAGE DAYS'] = df['COVERAGE DAYS'].astype(str)
df['COVERAGE PERIOD'] = df['COVERAGE DAYS'].str.split(' ').str[0]
df['COVERAGE PERIOD'] = pd.to_numeric(df['COVERAGE PERIOD'], errors = 'coerce')

c. Slice the dataframe to include strictly positive payment amounts and coverage period greater than zero

In [ ]:
mask2 = (df['AMOUNT'] > 0) & (df['COVERAGE PERIOD'] >= 0)
df2 = df.loc[mask2]

d. Find the standard deviation of COVERATE PERIOD

In [ ]:
std = statistics.stdev(df2['COVERAGE PERIOD'])
std

### Q3: What was the average annual expenditure with a 'START DATE' date between January 1, 2010 and December 31, 2016 (inclusive)? Only consider payments wiht strictly positive amounts.

a. Set start and end values

In [ ]:
START = '2010-01-01'
END = '2016-12-31'

b. Slice dataframe to include strictly positive payment amounts and start dates between Jan 1, 2010 and Dec 31, 2016

In [ ]:
mask3 = (df['AMOUNT'] > 0) & (df_pos['START_DATETIME'] >= START) & (df_pos['START_DATETIME'] <= END)
df3 = df.loc[mask3]

In [ ]:
df_mean = df3.groupby('START YEAR')['AMOUNT'].sum()
df_mean = df_mean.reset_index()
df_mean.columns = [['year','yearly amount']]

In [ ]:
df_mean['yearly amount'].mean()

### Q4: Find the 'OFFICE' with the highest total expenditures with a 'START DATE' in 2016. For this office, find the 'PURPOSE' that accounts for the highest total expenditures. What fraction of the total expenditures (all records, all offices) with a 'START DATE' in 2016 do these expenditures amount to?

a. Find the 'OFFICE' with the highest total expenditures with a 'START DATE' in 2016. 

In [ ]:
mask4 = (df['START YEAR'] == 16.0) & (df['AMOUNT'] > 0)
df4 = df.loc[mask4]

In [ ]:
df4o = df4.groupby('OFFICE', sort = True)['AMOUNT'].sum().reset_index()

In [ ]:
df4o.sort_values('AMOUNT', ascending=False).max()

In [ ]:
df4o.iloc[550]

In [ ]:
office = 'GOVERNMENT CONTRIBUTIONS'

b. For this office, find the 'PURPOSE' that accounts for the highest total expenditures.

In [ ]:
mask4 = (df4['OFFICE'] == office)
df4gc = df4.loc[mask4]

In [ ]:
df4gcp = df4gc.groupby('PURPOSE', sort = True)['AMOUNT'].sum().reset_index()

In [ ]:
df4gcp.iloc[6]

In [ ]:
purpose = 'FERS'

c. What fraction of the total expenditures (all records, all offices) with a 'START DATE' in 2016 do these expenditures amount to?

In [ ]:
total_exp16 = df4['AMOUNT'].sum()
total_exp16

In [ ]:
mask4 = (df4['OFFICE'] == office) & (df4['PURPOSE'] == purpose)
df4gcp = df4.loc[mask4]

In [ ]:
office_purpose_exp16 = df4gcp['AMOUNT'].sum()
office_purpose_exp16

In [ ]:
fraction = office_purpose_exp16/total_exp16
fraction

### Q5: What was the highest average staff salary among all representatives in 2016? Assume staff sizes is equal to the number of unique payees in the 'PERSONNEL COMPENSATION' category for each representative.

a. Slice dataframe to include PERSONNEL COMPENSATION payments in 2016

In [ ]:
mask5 = (df['CATEGORY'] == 'PERSONNEL COMPENSATION') & (df['START YEAR'] == 16.0)
df5 = df.loc[mask5]

b. Take the total amount paid to unique payees by office

In [ ]:
df5s = df5.groupby(['OFFICE','PAYEE'])['AMOUNT'].sum().reset_index()

c. Count number of payees by office

In [ ]:
df5s['STAFF SIZE'] = 1
df5ss = df5s.groupby(['OFFICE'])['AMOUNT', 'STAFF SIZE'].sum().reset_index()

d. Take the average amount paid to all payees by office

In [ ]:
df5ss ['AVG STAFF SALARY'] = df5ss['AMOUNT']/df5ss['STAFF SIZE']
df5ss.sort_values(by='AVG STAFF SALARY', ascending = False)
df5ss.max()

### Q6: What was the median rate of annual turnover in staff between 2011 and 2016 (inclusive)? Turnover for 2011 should be calculated as the fraction of a representative's staff from 2010 who did not carry over to 2011. Only consider representatives who served for at least 4 years and had staff size of at least 5 every year that they served.

a. Dataframe for question 6 looks at payees as employees

In [171]:
df['OFFICE']=df['OFFICE'].str.rstrip('-')

In [172]:
df['OFFICE']=df['OFFICE'].str.rstrip('--')

In [173]:
mask6 = (df['CATEGORY'] == 'PERSONNEL COMPENSATION')
df6 = df.loc[mask6]
df6 = df6[['OFFICE','PAYEE','START YEAR']]
df6.head()

,OFFICE,PAYEE,START YEAR
86,OFFICE OF THE SPEAKER,"DALY, BRENDAN",9.0
87,OFFICE OF THE SPEAKER,"LEVISON, LARA K",9.0
88,OFFICE OF THE SPEAKER,"ONEK,JOSEPH",9.0
89,OFFICE OF THE SPEAKER,"PARKER, WYNDEE R",9.0
90,OFFICE OF THE SPEAKER,"PRICE, REVA B",9.0


b. Dataframe s groups staff by start year

In [174]:
df6s = df6.groupby(['OFFICE','PAYEE','START YEAR']).count().reset_index()
df6s = df6s.drop_duplicates()
df6s.head()

,OFFICE,PAYEE,START YEAR
0,19TH DISTRICT OF FLORIDA,"ALVI,HINA R",9.0
1,19TH DISTRICT OF FLORIDA,"BAFER, COURTNEY D",9.0
2,19TH DISTRICT OF FLORIDA,"BENSON, JILL A",9.0
3,19TH DISTRICT OF FLORIDA,"BERGER,LAUREN",9.0
4,19TH DISTRICT OF FLORIDA,"BRIER, THERESA K",9.0


In [175]:
df6s['START YEAR'] = df6s['START YEAR'].astype(int)
mask6 = (df6s['START YEAR'] >= 10) & (df6s['START YEAR'] <= 16)
df6s = df6s.loc[mask6]

In [176]:
df6s['START YEAR'].value_counts().sort_values()

14    13444
12    13949
15    15028
13    15149
11    17049
10    17325
16    23504
Name: START YEAR, dtype: int64

In [177]:
df6s['START YEAR'] = df6s['START YEAR'].astype(str)

In [178]:
df6s['START YEAR'] = df6s['START YEAR'].str.zfill(2)

In [179]:
df6s.columns

Index(['OFFICE', 'PAYEE', 'START YEAR'], dtype='object')

In [180]:
df6s['STAFF SIZE'] = 1
df6s.head()

,OFFICE,PAYEE,START YEAR,STAFF SIZE
24,2014 HON. ANDRE CARSON,9/1/14,14,1
25,"2014 HON. ROBERT C. ""BOBBY"" SCOTT",9/1/14,14,1
26,2014 OTHER ADMINISTRATION,9/1/14,14,1
27,2015 HON. KRISTI L. NOEM,5/1/15,15,1
28,2015 HON. RENEE L. ELLMERS,9/1/15,15,1


c. Dataframe ss gets number of staff in each startyear year by office

In [181]:
df6ss = df6s.groupby(['OFFICE', 'START YEAR'])['STAFF SIZE'].sum().reset_index()
df6ss.head()

,OFFICE,START YEAR,STAFF SIZE
0,2014 HON. ANDRE CARSON,14,1
1,"2014 HON. ROBERT C. ""BOBBY"" SCOTT",14,1
2,2014 OTHER ADMINISTRATION,14,1
3,2015 HON. KRISTI L. NOEM,15,1
4,2015 HON. RENEE L. ELLMERS,15,1


d. Dataframe st gets staff turnover in each office by startyear

In [182]:
df6s = pd.get_dummies(df6s, prefix=['year_'], columns=['START YEAR'])

In [183]:
df6s.tail()

,OFFICE,PAYEE,STAFF SIZE,year__10,year__11,year__12,year__13,year__14,year__15,year__16
167678,VETERANS' AFFAIRS,"WU, ARTHUR K",1,0,1,0,0,0,0,0
167680,VETERANS' AFFAIRS,"ZAPATA,JIAN",1,1,0,0,0,0,0,0
167681,VETERANS' AFFAIRS,"ZAPATA,JIAN",1,0,1,0,0,0,0,0
167682,VETERANS' AFFAIRS,"ZAPATA,JIAN",1,0,0,1,0,0,0,0
167683,VETERANS' AFFAIRS,"ZAPATA,JIAN",1,0,0,0,1,0,0,0


In [184]:
yrs = ['year__10','year__11', 'year__12','year__13','year__14','year__15','year__16']
df6st = df6s.groupby(['OFFICE', 'PAYEE'])[yrs].sum().reset_index()
df6st.head()

,OFFICE,PAYEE,year__10,year__11,year__12,year__13,year__14,year__15,year__16
0,2014 HON. ANDRE CARSON,9/1/14,0,0,0,0,1,0,0
1,"2014 HON. ROBERT C. ""BOBBY"" SCOTT",9/1/14,0,0,0,0,1,0,0
2,2014 OTHER ADMINISTRATION,9/1/14,0,0,0,0,1,0,0
3,2015 HON. KRISTI L. NOEM,5/1/15,0,0,0,0,0,1,0
4,2015 HON. RENEE L. ELLMERS,9/1/15,0,0,0,0,0,1,0


In [185]:
for i in range(11,17):
    turnover = 'turnover_'+str(i)
    df6st[turnover] = 0

e. Only consider offices of members who served for at least 4 years and had a staff size of at least 5 every year that they served 

e.i. Dataframe m5 filters members with staff size at least 5

In [186]:
df6m5 = df6st.groupby(['OFFICE'])[yrs].sum().reset_index()

df6m5['STAFF_TOT'] = df6m5['year__11']+df6m5['year__12']+df6m5['year__13']+df6m5['year__14']+df6m5['year__15']+df6m5['year__16']
df6m5['STAFF_TOT_GR5'] = 0
df6m5.loc[(df6m5['STAFF_TOT'] >=5), 'STAFF_TOT_GR5'] = 1 

In [187]:
df6m5['STAFF_TOT_GR5'].value_counts()

1    772
0    145
Name: STAFF_TOT_GR5, dtype: int64

e.ii. Dataframe m4 filters members serving at least 4 years

In [188]:
df6m4 = df[['OFFICE','START YEAR']]

In [189]:
df6m4.tail()

,OFFICE,START YEAR
3542016,FORD DC ENHANCE,15.0
3542017,FORD DC ENHANCE,15.0
3542018,FORD DC ENHANCE,15.0
3542019,FORD DC ENHANCE,15.0
3542020,PAGING,15.0


In [190]:
df6m4 = df6m4.groupby(['OFFICE','START YEAR']).count().reset_index()

In [191]:
df6m4.columns

Index(['OFFICE', 'START YEAR'], dtype='object')

In [192]:
df6m4['START YEAR'] = df6m4['START YEAR'].astype(int)

In [193]:
df6m4['START YEAR'] = df6m4['START YEAR'].astype(str)

In [194]:
df6m4['START YEAR'] = df6m4['START YEAR'].str.zfill(2)

In [161]:
#df6m4['START YEAR'].value_counts()

In [195]:
df6m4 = pd.get_dummies(df6m4, prefix=['year_'], columns=['START YEAR'])

In [163]:
df6m4.columns

Index(['OFFICE', 'year__00', 'year__01', 'year__02', 'year__03', 'year__04',
       'year__05', 'year__06', 'year__07', 'year__08', 'year__09', 'year__10',
       'year__11', 'year__12', 'year__13', 'year__14', 'year__15', 'year__16',
       'year__17', 'year__18', 'year__19', 'year__20', 'year__25', 'year__44',
       'year__51', 'year__61', 'year__98', 'year__99'],
      dtype='object')

In [196]:
years = ['year__07', 'year__08','year__09','year__10','year__11', 'year__12','year__13','year__14','year__15','year__16', 'year__17', 'year__18']
df6m4 = df6m4.groupby(['OFFICE'])[years].sum().reset_index()


In [197]:
df6m4.tail()

,OFFICE,year__07,year__08,year__09,year__10,year__11,year__12,year__13,year__14,year__15,year__16,year__17,year__18
2712,TECHNOLOGY SUPPORT,0,0,0,0,0,0,1,1,1,1,1,0
2713,TRANSITION ACITIVITES,1,1,1,1,0,0,0,0,0,0,0,0
2714,TRANSPORTATION-INFRASTRUCTURE,0,1,1,1,1,1,1,1,1,1,1,1
2715,VETERANS' AFFAIRS,0,0,1,1,1,1,1,1,1,1,1,0
2716,r,0,0,0,0,0,0,0,0,0,0,1,0


In [198]:
df6m4['YEARS_SERVED'] = df6m4.iloc[:, 1:].sum(1)

In [199]:
df6m4['YEARS_SERVED_GR4'] = 0
df6m4.loc[(df6m4['YEARS_SERVED'] >=4), 'YEARS_SERVED_GR4'] = 1 

In [200]:
df6m4['YEARS_SERVED_GR4'].value_counts()

0    1983
1     734
Name: YEARS_SERVED_GR4, dtype: int64

In [170]:
#df6m4['OFFICE'].value_counts()

e.iii. Dataframe m54 tags members fitting both criteria

In [201]:
df6m54 = pd.merge(df6m4, df6m5, on='OFFICE', how = 'outer')
df6m54['MEMBER_SCOPE'] = 0
df6m54.loc[(df6m54['YEARS_SERVED_GR4'] == 1) & (df6m54['STAFF_TOT_GR5']), 'MEMBER_SCOPE'] = 1 

In [202]:
df6m54['MEMBER_SCOPE'].sum()

663

f. Calculate turnover for 2011 as the fraction of a representative's staff from 2010 who did not carry over to 2011.

In [203]:
yrs = ['year__10','year__11', 'year__12','year__13','year__14','year__15','year__16']
df6st.loc[(df6st['year__10'] == 1) &  (df6st['year__11'] != 1), 'turnover_11'] = 1 
df6st.loc[(df6st['year__11'] == 1) &  (df6st['year__12'] != 1), 'turnover_12'] = 1 
df6st.loc[(df6st['year__12'] == 1) &  (df6st['year__13'] != 1), 'turnover_13'] = 1 
df6st.loc[(df6st['year__13'] == 1) &  (df6st['year__14'] != 1), 'turnover_14'] = 1 
df6st.loc[(df6st['year__14'] == 1) &  (df6st['year__15'] != 1), 'turnover_15'] = 1 
df6st.loc[(df6st['year__15'] == 1) &  (df6st['year__16'] != 1), 'turnover_16'] = 1 

In [204]:
df6st.tail()

,OFFICE,PAYEE,year__10,year__11,year__12,year__13,year__14,year__15,year__16,turnover_11,turnover_12,turnover_13,turnover_14,turnover_15,turnover_16
53524,VETERANS' AFFAIRS,"WIBLEMO,CATHLEEN C",1,1,1,1,1,1,1,0,0,0,0,0,0
53525,VETERANS' AFFAIRS,"WILKINS,DANIEL M",1,1,0,0,0,0,0,0,1,0,0,0,0
53526,VETERANS' AFFAIRS,"WILLIAMS, MEGAN",1,0,0,0,0,0,0,1,0,0,0,0,0
53527,VETERANS' AFFAIRS,"WU, ARTHUR K",1,1,0,0,0,0,0,0,1,0,0,0,0
53528,VETERANS' AFFAIRS,"ZAPATA,JIAN",1,1,1,1,0,0,0,0,0,0,1,0,0


In [205]:
df6st.columns

Index(['OFFICE', 'PAYEE', 'year__10', 'year__11', 'year__12', 'year__13',
       'year__14', 'year__15', 'year__16', 'turnover_11', 'turnover_12',
       'turnover_13', 'turnover_14', 'turnover_15', 'turnover_16'],
      dtype='object')

f.i. Dataframe tt gives total staff turnover in each year

In [206]:
# New dataframe with number of staff turned over each year
turns = ['turnover_11', 'turnover_12','turnover_13', 'turnover_14', 'turnover_15', 'turnover_16']
df6tt = df6st.groupby(['OFFICE'])[turns].sum().reset_index()
df6tt.tail()

,OFFICE,turnover_11,turnover_12,turnover_13,turnover_14,turnover_15,turnover_16
912,STANDARDS OF OFFICIAL CONDUCT,29,0,0,0,0,0
913,SUMMER CAMP,0,2,0,1,1,5
914,TECHNICAL ASSISTANTS,0,1,0,0,1,0
915,TRANSPORTATION-INFRASTRUCTURE,21,46,15,31,25,27
916,VETERANS' AFFAIRS,6,23,6,12,9,9


f.ii. New dataframe is shaped long by office and year with staff turnover and total staff as columns

In [207]:
df6ty = pd.DataFrame()
for col in df6tt:
    if col != 'OFFICE':
        df_new = df6tt[['OFFICE', col]]
        df_new['START YEAR'] = col
        df_new['TURNOVER'] = df_new[col]
        df_new = df_new[['OFFICE', 'START YEAR','TURNOVER']]
        frames = [df6ty, df_new]
        df6ty = pd.concat(frames)
df_new.head()

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,OFFICE,START YEAR,TURNOVER
0,2014 HON. ANDRE CARSON,turnover_16,0
1,"2014 HON. ROBERT C. ""BOBBY"" SCOTT",turnover_16,0
2,2014 OTHER ADMINISTRATION,turnover_16,0
3,2015 HON. KRISTI L. NOEM,turnover_16,1
4,2015 HON. RENEE L. ELLMERS,turnover_16,1


In [208]:
df6ty.tail()

,OFFICE,START YEAR,TURNOVER
912,STANDARDS OF OFFICIAL CONDUCT,turnover_16,0
913,SUMMER CAMP,turnover_16,5
914,TECHNICAL ASSISTANTS,turnover_16,0
915,TRANSPORTATION-INFRASTRUCTURE,turnover_16,27
916,VETERANS' AFFAIRS,turnover_16,9


In [209]:
df6ty.loc[(df6ty['START YEAR'] == 'turnover_10'), 'START YEAR'] = '10'
df6ty.loc[(df6ty['START YEAR'] == 'turnover_11'), 'START YEAR'] = '11'
df6ty.loc[(df6ty['START YEAR'] == 'turnover_12'), 'START YEAR'] = '12'
df6ty.loc[(df6ty['START YEAR'] == 'turnover_13'), 'START YEAR'] = '13'
df6ty.loc[(df6ty['START YEAR'] == 'turnover_14'), 'START YEAR'] = '14'
df6ty.loc[(df6ty['START YEAR'] == 'turnover_15'), 'START YEAR'] = '15'
df6ty.loc[(df6ty['START YEAR'] == 'turnover_16'), 'START YEAR'] = '16'

In [210]:
df6ty['START YEAR'].dtype

dtype('O')

In [211]:
df6ty.tail().sort_values(by="OFFICE")

,OFFICE,START YEAR,TURNOVER
912,STANDARDS OF OFFICIAL CONDUCT,16,0
913,SUMMER CAMP,16,5
914,TECHNICAL ASSISTANTS,16,0
915,TRANSPORTATION-INFRASTRUCTURE,16,27
916,VETERANS' AFFAIRS,16,9


f.iii. Dataframe stt54 give staff turnover by year for each office that fits criteria of years served and staff size

In [214]:
cols = ['OFFICE', 'START YEAR']
df6stt = pd.merge(df6ty, df6ss, on = ['OFFICE', 'START YEAR'], how = 'outer')

In [215]:
len(df6stt['OFFICE'])

6110

In [216]:
df6stt54 = pd.merge(df6stt, df6m54[['OFFICE', 'MEMBER_SCOPE']], on = ['OFFICE'], how = 'left')

In [219]:
len(df6stt54['OFFICE'])

6110

In [218]:
df6stt54 = df6stt54.drop_duplicates()

In [221]:
df6stt54['MEMBER_SCOPE'].value_counts()

1    4422
0    1688
Name: MEMBER_SCOPE, dtype: int64

In [222]:
mask6f = (df6stt54['MEMBER_SCOPE'] == 1)
df6stt54 = df6stt54.loc[mask6f]

In [223]:
df6stt54.tail()

,OFFICE,START YEAR,TURNOVER,STAFF SIZE,MEMBER_SCOPE
6102,REPUBLICAN POLICY COMMITTEE,10,NaN,10.0,1
6105,SERGEANT AT ARMS,10,NaN,116.0,1
6107,TECHNICAL ASSISTANTS,10,NaN,3.0,1
6108,TRANSPORTATION-INFRASTRUCTURE,10,NaN,98.0,1
6109,VETERANS' AFFAIRS,10,NaN,40.0,1


In [224]:
df6stt['TURN RATE'] = df6stt['TURNOVER']/df6stt['STAFF SIZE']

In [225]:
df6stt['TURN RATE'].median()

0.21428571428571427